In [1]:
import numpy as np

# Qiskit Certification Prep - Module 1.1: Define Pauli Operators

## Section 1.1.0 Pauli class

- Focus: `qiskit.quantum_info.Pauli` (tensor products of single-qubit Paulis with
  a global phase).
- IBM Docs: https://quantum.cloud.ibm.com/docs/en/api/qiskit/qiskit.quantum_info.Pauli
- Methods we will cover:  
  - `to_matrix`: unitary matrix representation.  
  - `to_instruction`: circuit `Instruction` for `QuantumCircuit`.  
  - `compose`: order-aware product to combine Paulis.  
- Additional class: `qiskit.quantum_info.SparsePauliOp` for sparse linear
  combinations of Paulis (e.g., Hamiltonians/observables).  
- Goal: translate between strings ↔ matrices ↔ circuit instructions, compose
  Paulis, and use `SparsePauliOp` efficiently.  

## Section 1.1.1: Review from Module 0.1
### Pauli Gates
Pauli gates are the basic quantum logic gates. Their matrix representations and actions are:

- **X (NOT gate)**:
  $X = \begin{bmatrix} 0 & 1 \\ 1 & 0 \end{bmatrix}$
  Swaps $|0⟩$ and $|1⟩$

- **Y**:
  $Y = \begin{bmatrix} 0 & -i \\ i & 0 \end{bmatrix}$
  Applies a bit and phase flip

- **Z**:
  $Z = \begin{bmatrix} 1 & 0 \\ 0 & -1 \end{bmatrix}$
  Flips the phase of $|1⟩$

  

### Properties of the Hadamard Gate and Pauli Gates:
For $\sigma_i \in \{H, X, Y, Z\}$
- **Involutory**: $\sigma_i^2 =I$
- **Traceless**: $\text{Tr} (\sigma_i) =0$
- **Hermitian**: $\sigma_i = \sigma_i^\dagger$
- **Unitary**: $\sigma_i^\dagger \sigma_i = I$

### Common Identities:
- $X = HZH$
- $Z = HXH$
- $H^2 = I$
- $X^2 = Y^2 = Z^2 = I$
- $XYZ = iI$  → So $-iXYZ = I$

### Commutation / Anticommutation:
- $ZX = -XZ$
- $YX = -XY$
- $ZY = -YZ$

These identities arise from the non-commutative nature of quantum  
gates and are fundamental to quantum algorithms and error correction.

## Quick Aside on Numpy syntax
### NumPy syntax — `np.vdot` vs `@` (matrix multiplication)

- `@` is Python’s matrix-multiplication operator. For NumPy arrays it behaves
  like `np.matmul`:
  - Shapes: `(m, n) @ (n, p) → (m, p)`, `(n,) @ (n,) → scalar`.
  - For 1-D: `(n,) @ (n, p) → (p,)` (row-vector times matrix),  
    `(m, n) @ (n,) → (m,)` (matrix times column-vector).
  - No implicit complex conjugation is applied.

- `np.vdot(a, b)` computes the **conjugate dot product**
  of the flattened
  inputs: `conj(a.ravel()) · b.ravel() → scalar`.

- Quick rule of thumb:
  - Use `@` (or `np.matmul`) for matrix–vector / matrix–matrix products.
  - Use `np.vdot` for inner products that should   **conjugate the first input**.

### Runnable examples

```python
import numpy as np

# Complex vectors
a = np.array([1+2j, 3-1j])
b = np.array([2-1j, -1+4j])

# 1) Inner products
print("a @ b =", a @ b)                 # NO conjugation
print("np.dot(a, b) =", np.dot(a, b))   # same as a @ b for 1-D
print("np.vdot(a, b) =", np.vdot(a, b)) # conjugates 'a' and flattens both


## Section 1.1.2 `Pauli.to_matrix` — quick explainer + example

- What it does: `Pauli.to_matrix()` returns the dense unitary matrix
  (NumPy `ndarray`) of a `Pauli` on $n$ qubits (shape $2^n \times 2^n$).  
- When to use:  
  - To verify algebra (e.g., compare with a Kronecker product you built).  
  - To compute overlaps/expectations with states (e.g., for small $n$).  
- Notes: preserves the global phase encoded in the `Pauli`.  



In [2]:
### Minimal examples

from qiskit.quantum_info import Pauli, Statevector
import numpy as np

# 1) Single-qubit example: check the matrix of 'Y'
p_y = Pauli('Y')
M_y = p_y.to_matrix()
print("Y matrix:\n", M_y)

# Cross-check with the known matrix
Y_known = np.array([[0, -1j], [1j, 0]])
print("Matches known Y?", np.allclose(M_y, Y_known))

# 2) Two-qubit example: 'XZ' equals X ⊗ Z
p_xz = Pauli('XZ')
M_xz = p_xz.to_matrix()
X = np.array([[0, 1], [1, 0]])
Z = np.array([[1, 0], [0, -1]])
print("\nXZ equals X⊗Z ?", np.allclose(M_xz, np.kron(X, Z)))

# 3) Use case: expectation ⟨+| Z |+⟩ = 0
plus = Statevector.from_label('+')       # |+> = (|0> + |1>)/√2
p_z = Pauli('Z')
M_z = p_z.to_matrix()
exp_plus_Z = (plus.data.conj().T @ M_z @ plus.data).real
print("\nExpectation <+|Z|+>:", exp_plus_Z)



Y matrix:
 [[0.+0.j 0.-1.j]
 [0.+1.j 0.+0.j]]
Matches known Y? True

XZ equals X⊗Z ? True

Expectation <+|Z|+>: -2.2371143170757382e-17


## Section 1.1.3 `Pauli.to_instruction` — quick explainer + use case

- What it does: converts a `Pauli` into a circuit-level `Instruction`.  
- Why it’s useful: lets you drop a multi-qubit Pauli directly into a
  `QuantumCircuit`, then rely on transpilation to map it to the target backend.  
- Returns: an `Instruction` with the correct qubit count and global phase.  
- Typical workflow: build `Pauli` → `to_instruction()` → `QuantumCircuit.append`
  → `transpile` for your backend.  


The following example combines a few concepts we've seen before:
- Qubit ordering convention in Qiskit
- Rz(pi) is the same as the Pauli Z matrix
- How to apply `to_instruction` and `evolve` methods in a quantum circuit

In [10]:
### Minimal example + backend mapping

from qiskit.quantum_info import Pauli, Statevector
from qiskit import QuantumCircuit, transpile
from qiskit_ibm_runtime.fake_provider import FakeSherbrooke

# 1) Build a 2-qubit Pauli and convert to an Instruction
p = Pauli("XZ")                # acts as X on q0 and Z on q1
instr = p.to_instruction()
print(type(instr), instr.num_qubits, instr.name)

# 2) Use it in a circuit
qc = QuantumCircuit(2, name="pauli_demo")
qc.append(instr, [0, 1])
print(qc)

# 3) Functional check: equals explicit X on q1 and Z on q0
qc_explicit = QuantumCircuit(2)
qc_explicit.x(1)
qc_explicit.z(0)

sv_in = Statevector.from_label("00")
sv_pauli = sv_in.evolve(qc)
sv_explicit = sv_in.evolve(qc_explicit)
print("Same final state?", sv_pauli.equiv(sv_explicit))

# 4) Transpile onto a (fake) backend to see native decomposition
backend = FakeSherbrooke()
t_qc = transpile(qc, backend)
print(t_qc)


<class 'qiskit.circuit.library.generalized_gates.pauli.PauliGate'> 2 pauli
     ┌────────────┐
q_0: ┤0           ├
     │  Pauli(XZ) │
q_1: ┤1           ├
     └────────────┘
Same final state? True
global phase: π/2
          ┌───────┐
q_0 -> 60 ┤ Rz(π) ├
          └─┬───┬─┘
q_1 -> 61 ──┤ X ├──
            └───┘  


## Section 1.1.4 `Pauli.compose` — quick explainer + use cases

- What it does: algebraic product of two Paulis, returning a new `Pauli`.  
- Order matters: by default `self.compose(other)` means `self @ other`.  
- Use `front=True` to flip order: computes `other @ self`.  
- Phases are tracked: products can acquire factors in `{±1, ±i}`.  
- Supports subsets with `qargs` for multi-qubit targeting.  


In [11]:
### Minimal examples (single-qubit)

from qiskit.quantum_info import Pauli

X, Y, Z, I = Pauli("X"), Pauli("Y"), Pauli("Z"), Pauli("I")

# Default order: self @ other
XZ = X.compose(Z)                 # equals XZ = -i Y
ZX = Z.compose(X)                 # equals ZX = +i Y

print("X ∘ Z  ->", XZ)            # shows phase and label, e.g. '-iY'
print("Z ∘ X  ->", ZX)            # e.g. 'iY'

# Sanity check: X ∘ X = I (no phase)
print("X ∘ X  ->", X.compose(X))  # 'I'


X ∘ Z  -> iY
Z ∘ X  -> -iY
X ∘ X  -> I


### Section 1.1.5: `tensor` (Common confusion: `compose` vs `tensor`)

## 1) Conceptual overview  

**Pauli operators.** The single-qubit Paulis are $I, X, Y, Z$ with  
$X^2=Y^2=Z^2=I$ and $XY=iZ$, $YZ=iX$, $ZX=iY$. Pauli objects in Qiskit track a  
global phase $i^k$ with $k\in\{0,1,2,3\}$.  

**Tensor product (`tensor`).** Given two Paulis $P$ and $Q$,  
$P\otimes Q$ builds a larger operator that acts on the combined register.  
It increases the number of qubits by $\mathrm{qubits}(P)+\mathrm{qubits}(Q)$.  
No qubit overlap is required.  

**Composition (`compose`).** `compose` multiplies Pauli operators on the same  
qubits (or a chosen subset via `qargs`). By default it computes `self @ other`  
(matrix product), accumulating any overall phase. It does **not** change the  
number of qubits unless you use `qargs` to target a subset within a larger Pauli.  

**Rule of thumb.**  

- Use `tensor` to **stack** Pauli factors on **disjoint** sets of qubits.  
- Use `compose` to **multiply** Paulis on the **same** qubits (or on specified  
  `qargs` inside a larger register).

##  Hands‑on examples (run these)  

The examples below use `qiskit.quantum_info.Pauli`. We print labels, phases,  
and matrices so you can see the differences between `tensor` and `compose`.

In [5]:
from qiskit.quantum_info import Pauli

# Single-qubit Paulis
pX = Pauli("X")
pZ = Pauli("Z")

print("=== Example 1: tensor builds a larger, disjoint operator ===")
t = pX.tensor(pZ)   # P ⊗ Q
print("tensor label:", t.to_label(), " | num_qubits:", t.num_qubits)
print(t.to_matrix())

print("\n=== Example 2: compose multiplies on the same qubit(s) ===")
c = pX.compose(pZ)  # X @ Z  (same single qubit)
print("compose label:", c.to_label(), " | phase:", c.phase)
print(c.to_matrix())

print("\nNote: `phase` is stored as multiples of i: phase k means factor i**k.")

=== Example 1: tensor builds a larger, disjoint operator ===
tensor label: XZ  | num_qubits: 2
[[ 0.+0.j  0.+0.j  1.+0.j  0.+0.j]
 [ 0.+0.j  0.+0.j  0.+0.j -1.+0.j]
 [ 1.+0.j  0.+0.j  0.+0.j  0.+0.j]
 [ 0.+0.j -1.+0.j  0.+0.j  0.+0.j]]

=== Example 2: compose multiplies on the same qubit(s) ===
compose label: iY  | phase: 3
[[ 0.+0.j  1.+0.j]
 [-1.+0.j  0.+0.j]]

Note: `phase` is stored as multiples of i: phase k means factor i**k.


## Section 1.1.6  Multiple Choice Questions

**Q1.** *Given the code below, what is the printed output?*
```python
from qiskit.quantum_info import Pauli
import numpy as np

X, Y, Z = Pauli("X"), Pauli("Y"), Pauli("Z")

lhs = X.compose(Z).to_matrix()      # XZ
rhs = (-1j) * Y.to_matrix()         # -i · Y
print(np.allclose(lhs, rhs))

A) True

B) False

C) Raises a TypeError

D) Prints a complex 2×2 matrix
<details> <summary><b>Answer (click to expand)</b></summary>

A) True


Explanation:
Pauli algebra gives $\sigma_X \sigma_Z = -i,\sigma_Y$. Thus
X.compose(Z).to_matrix() equals (-1j) * Y.to_matrix(), so
np.allclose(lhs, rhs) prints True.

Why others are incorrect:

B: The matrices do match up to the correct global phase $-i$.

C: compose and to_matrix() are valid for Pauli; no error.

D: The code prints a boolean, not a matrix.


**Q2.** *Given the code below, what is the printed output?*
```python
from qiskit.quantum_info import Statevector, Pauli
import numpy as np

sv = Statevector.from_label('+')       # |+> = (|0> + |1>)/√2
Z = Pauli('Z').to_matrix()

val = sv.data.conj().T @ Z @ sv.data   # quadratic form ⟨+|Z|+⟩ using .data
print(val)

```

A) 0j

B) 1+0j

C) Raises a TypeError because .data is not a NumPy array

D) Prints a 2×2 matrix

<details> <summary><b>Answer (click to expand)</b></summary>
A) 0j

</details> <details> <summary><b>Explanation (click to expand)</b></summary>
Statevector.data is a NumPy array of amplitudes, so @ does standard
matrix–vector multiplication with Z.

The expectation $\langle + | Z | + \rangle = 0$, and the computation returns
a complex scalar 0j.

Why others are incorrect:

B: $\langle + | Z | + \rangle \neq 1$.

C: .data is a NumPy array; no type error occurs.

D: The expression evaluates to a scalar, not a matrix.

</details> ``` ::contentReference[oaicite:0]{index=0}

**Q3.** Which code snippet most likely produced this output?
```
(1+0j)
```

A)
```
from qiskit.quantum_info import Statevector, Pauli
sv = Statevector.from_label('0')
Z = Pauli('Z').to_matrix()
print(sv.data.conj().T @ Z @ sv.data)
```

B)

```
from qiskit.quantum_info import Statevector, Pauli
sv = Statevector.from_label('+')
Z = Pauli('Z').to_matrix()
print(sv.data.conj().T @ Z @ sv.data)
```

C)

```
from qiskit.quantum_info import Statevector, Pauli
sv = Statevector.from_label('0')
X = Pauli('X').to_matrix()
print(sv.data.conj().T @ X @ sv.data)
```

D)

```
from qiskit.quantum_info import Statevector, Pauli
sv = Statevector.from_label('1')
Z = Pauli('Z').to_matrix()
print(sv.data.conj().T @ Z @ sv.data)
```

<details> <summary><b>Answer (click to expand)</b></summary>
A

</details> <details> <summary><b>Explanation (click to expand)</b></summary>
In A, the value is $\langle 0|Z|0\rangle = +1 \Rightarrow (1+0j)$.

B: $\langle +|Z|+\rangle = 0 \Rightarrow 0j$.

C: $\langle 0|X|0\rangle = 0 \Rightarrow 0j$.

D: $\langle 1|Z|1\rangle = -1 \Rightarrow (-1+0j)$.

**Q4. Which code snippet most likely produced this output?**

(0.5+0j)

A)

```
from qiskit.quantum_info import Statevector, SparsePauliOp
import numpy as np

sv = Statevector.from_label('++').data
H = SparsePauliOp.from_list([('XX', 0.5)]).to_matrix()
print(np.vdot(sv, H @ sv))
```

B)

```
from qiskit.quantum_info import Statevector, SparsePauliOp
import numpy as np

sv = Statevector.from_label('++').data
H = SparsePauliOp.from_list([('ZZ', 0.5)]).to_matrix()
print(np.vdot(sv, H @ sv))
```

C)

```
from qiskit.quantum_info import Statevector, Pauli
import numpy as np

sv = Statevector.from_label('00').data
M = Pauli('XX').to_matrix()
print(np.vdot(sv, M @ sv))
```

D)

```
from qiskit.quantum_info import Statevector, Pauli
import numpy as np

sv = Statevector.from_label('00').data
M = Pauli('ZI').to_matrix()
print(np.vdot(sv, M @ sv))
```

<details> <summary><b>Answer (click to expand)</b></summary>
A

</details> <details> <summary><b>Explanation (click to expand)</b></summary>
In A, $\langle ++ | \tfrac{1}{2} X!\otimes!X | ++ \rangle
= \tfrac{1}{2}\cdot 1 = 0.5$, since $|+\rangle$ is a $+1$ eigenstate of $X$.

B: $\langle ++ | ZZ | ++ \rangle = 0$ (each $|+\rangle$ is unbiased in $Z$).

C: $\langle 00 | XX | 00 \rangle = 0$ (off-diagonal flips yield zero
expectation on a computational basis state).

D: $\langle 00 | ZI | 00 \rangle = 1$ (not $0.5$).


**Q5.** *Given the code below, what is the printed output?*

```python
from qiskit.quantum_info import Pauli, Statevector
from qiskit import QuantumCircuit

p = Pauli("XZ")                  
instr = p.to_instruction()

qc = QuantumCircuit(2)
qc.append(instr, [0, 1])

sv = Statevector.from_label("00").evolve(qc)
print(sv.to_dict())              
```

A) {'00': 1.0}

B) {'01': 1.0}

C) {'10': 1.0}

D) {'11': 1.0}

<details> <summary><b>Answer (click to expand)</b></summary>

C) {'10': 1.0}

</details> <details> <summary><b>Explanation (click to expand)</b></summary>
Pauli("XZ").to_instruction() acts as X on qubit 0 and Z on qubit 1.

Starting from |00⟩, X flips qubit 0 → |10⟩.

Z on |0⟩ leaves it unchanged, so the final state is |10⟩, i.e.
{'10': 1.0}.

Why others are incorrect:

A: No gate leaves |00⟩ unchanged here (X flips q0).

B: Would require flipping q1, not q0.

D: Would require flipping q0 and adding a phase on q1=1 (not the case).

**Q6.** 
```
from qiskit import QuantumCircuit
from qiskit.quantum_info import Pauli

qc = QuantumCircuit(2)
p = Pauli("YI")   # Y on q0, I on q1
```

Which line correctly inserts the Pauli as an instruction on qubits [0, 1]?

A)

```
qc.append(p.to_instruction(), [0, 1])
```

B)

```
p.to_instruction(qc)
```

C)

```
qc.to_instruction(p)
```

D)

```
qc.append(p.to_instruction([0, 1]))
```

<details> <summary><b>Answer (click to expand)</b></summary>
A

</details> <details> <summary><b>Explanation (click to expand)</b></summary>
A (correct): Pauli.to_instruction() takes no arguments and returns an
Instruction. You then place it with qc.append(..., qubits).

B: Incorrect — .to_instruction does not accept a circuit argument.

C: Incorrect — QuantumCircuit has no .to_instruction method taking a
Pauli.

D: Incorrect — qubits belong to qc.append, not to .to_instruction.

We haven't covered the `Statevector` class yet, but for this next question, you should know that `probabilities_dict` prints the probabilities of each state

**Q7.** 
*What does the code print (probability distribution)?*
```
from qiskit import QuantumCircuit
from qiskit.quantum_info import Statevector, Pauli

qc = QuantumCircuit(2)
qc.h(0)                                    
qc.append(Pauli("ZI").to_instruction(), [0, 1])  
qc.append(Pauli("IX").to_instruction(), [0, 1]) 

sv = Statevector.from_label("00").evolve(qc)
print(sv.probabilities_dict())
```

A) {'01': 0.5, '11': 0.5}

B) {'00': 0.5, '10': 0.5}

C) {'01': 1.0}

D) {'00': 0.25, '01': 0.25, '10': 0.25, '11': 0.25}

<details> <summary><b>Answer (click to expand)</b></summary>

A) {'01': 0.5, '11': 0.5}

</details> <details> <summary><b>Explanation (click to expand)</b></summary>

After h(0): state is $(|00\rangle + |10\rangle)/\sqrt{2}$.

Pauli("ZI") applies Z to qubit 0, adding a phase to basis states
with q0 = 1. The state becomes $(|00\rangle - |10\rangle)/\sqrt{2}$.
Probabilities are unchanged by this global sign on the |10⟩ component.

Pauli("IX") flips qubit 1: $|00\rangle \to |01\rangle$, $|10\rangle \to |11\rangle$.
Final state: $(|01\rangle - |11\rangle)/\sqrt{2}$ → probabilities
{'01': 0.5, '11': 0.5}.

Why others are incorrect:

B: This is before applying IX; after the flip, support moves to 01 and 11.

C: Amplitudes split equally; probability 1.0 is incorrect.

D: The state is not uniform over four outcomes.



**Q8.** For single‑qubit Paulis `p = Pauli("X")` and `q = Pauli("Z")`, which call  
**necessarily** increases the number of qubits?  

A. `p.compose(q)`  
B. `p.tensor(q)`  
C. `q.compose(p, front=True)`  
D. `p.compose(q, qargs=[0])` 

<details>
<summary><b>Answer </b></summary>
B
<details>
<summary><b>Explanation</b></summary>

- `tensor` computes the Kronecker product $P\otimes Q$, which **stacks** the
  operators on disjoint registers and therefore **increases** the number of
  qubits by `num_qubits(P)+num_qubits(Q)`.
- `compose` multiplies Paulis on the **same** targets (or specified `qargs`),
  updating only the **phase/labels** and not the register size. Options like
  `front=True` change order of multiplication, not the size.

Hence, only `p.tensor(q)` necessarily increases qubit count.
</details>

- **Why Q8 is B?**  `tensor` performs the Kronecker product $P\otimes Q$, which **stacks** operators on disjoint registers and therefore increases the number of qubits. `compose` multiplies operators on the **same** qubits (or on selected `qargs`) and thus keeps the register size unchanged regardless of `front=True` or which subset you target.